In [2]:
import json
import re
import time
import requests
import numpy as np
import pandas as pd
import os
import glob
from tqdm.auto import tqdm
from datetime import datetime
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import warnings
warnings.filterwarnings('ignore')
from random import choice

In [3]:
current_datetime = datetime.now().strftime("%d-%m-%Y")
month_now = datetime.now().month
day_now = datetime.now().day

In [4]:
# proxy_list = []
# with open(r'C:\Users\Trenkin.Sergey\Downloads\Proxy-23-01-2024.txt','r') as reader :
#     for line in reader :
#         proxy = line.split('\n', 1)[0]
#         http_proxy =  'socks5://'+proxy
#         https_proxy = 'socks5://'+proxy
#         proxy_list.append({
#             'http': http_proxy,
#             'https': https_proxy,
#         })

# Cookies and Headers для парсинга скидки и цены

In [5]:
cookies_discount = {
    'tmr_lvid': '286726132090325c59b36aa6b5e4aeaa',
    'tmr_lvidTS': '1692163620235',
    '_ym_uid': '1692163620496112993',
    '_ym_d': '1692163620',
    'city_id': '118',
    'city_name': '%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3',
    'city_full_name': '%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3',
    'region_id': '090a0c31-3b91-49a3-aa35-0c784898170b',
    'region_name': '%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3',
    'region_subdomain': 'spb',
    '_tt_enable_cookie': '1',
    '_ttp': 'tbKEEA3IkGrgdKpaT6-HRNf0K8h',
    'ccart': 'off',
    'uxs_uid': '898ab7e0-3bf5-11ee-8efe-f5b770db12a1',
    'loggedin': '%D0%AE%D1%80%D0%B8%D0%B9',
    'cseid': 'f4935df6bdac0029832e99cef94a5bc2',
    'sessionid': 'nr24mlml3ze25rk734gkkmgx3yv9ou7x',
    'authorization': 'true',
    'mindboxDeviceUUID': 'c254fa3f-cfcd-4146-9778-65be21c6a4df',
    'directCrm-session': '%7B%22deviceGuid%22%3A%22c254fa3f-cfcd-4146-9778-65be21c6a4df%22%7D',
    'auid': '651660ce-a06e-4b17-bd6f-69feaf135b08:1r5lu8:WYtTZugo2xVE6SqAc5ZSp_W-n1s5ztPHYxPs8YIwGQA',
    '_ga_G74SVHM1ZG': 'GS1.2.1700824600.19.0.1700824600.60.0.0',
    '_ga_GRDJMMMRHQ': 'GS1.2.1700824600.33.0.1700824601.59.0.0',
    '_ga_GRSMPDQ6SE': 'GS1.2.1700824598.88.1.1700825516.60.0.0',
    '_ga_M9ZHPEJXBM': 'GS1.2.1700824599.88.1.1700825517.60.0.0',
    'qr-go-to-the-app-banner': '1',
    'sncid': 'b36ee46b-981d-11e5-add2-0050568f2465:1rBa43:O1Wy8KuwGDddSCM7NMHFsjGhdPaqxcjhdblvWWbVams',
    'clt': 'new_old',
    '_ym_isad': '2',
    '_gid': 'GA1.2.2131018395.1702274408',
    'city_auto_popup_shown': '1',
    '_ym_visorc': 'b',
    '_ga': 'GA1.2.333284150.1692163620',
    '_ga_HJNSJ6NG5J': 'GS1.1.1702276831.120.1.1702280271.20.0.0',
    'digi_uc': 'W1sidiIsIjM0OTMxNCIsMTcwMjI4MTIxMjA0Nl0sWyJ2IiwiODQwMjIiLDE3MDIyODAyMzEzNjZdLFsidiIsIjQ2Nzk3IiwxNzAyMjgwMTI4Mzc3XSxbInYiLCI5OTM3NTQiLDE3MDIyNzgwMTczMjddLFsidiIsIjE0NjUyMSIsMTcwMjI3NzI5NDExM10sWyJ2IiwiOTI2ODAiLDE3MDIyNzY5NDQ1NDRdLFsidiIsIjg4MjEzIiwxNzAxODU5MjE3OTg0XSxbInYiLCI5MTAyMSIsMTcwMTg1OTA2ODA2MF0sWyJ2IiwiOTg3MDUwIiwxNzAxNzc4NDc0NzY5XSxbInYiLCIyNTYyMTciLDE3MDEzNDIzNTc4NTRdXQ==',
}

headers_discount = {
    'AB-TESTS': '{}',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Type': 'application/json',
    # 'Cookie': 'tmr_lvid=286726132090325c59b36aa6b5e4aeaa; tmr_lvidTS=1692163620235; _ym_uid=1692163620496112993; _ym_d=1692163620; city_id=118; city_name=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3; city_full_name=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3; region_id=090a0c31-3b91-49a3-aa35-0c784898170b; region_name=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3; region_subdomain=spb; _tt_enable_cookie=1; _ttp=tbKEEA3IkGrgdKpaT6-HRNf0K8h; ccart=off; uxs_uid=898ab7e0-3bf5-11ee-8efe-f5b770db12a1; loggedin=%D0%AE%D1%80%D0%B8%D0%B9; cseid=f4935df6bdac0029832e99cef94a5bc2; sessionid=nr24mlml3ze25rk734gkkmgx3yv9ou7x; authorization=true; mindboxDeviceUUID=c254fa3f-cfcd-4146-9778-65be21c6a4df; directCrm-session=%7B%22deviceGuid%22%3A%22c254fa3f-cfcd-4146-9778-65be21c6a4df%22%7D; auid=651660ce-a06e-4b17-bd6f-69feaf135b08:1r5lu8:WYtTZugo2xVE6SqAc5ZSp_W-n1s5ztPHYxPs8YIwGQA; _ga_G74SVHM1ZG=GS1.2.1700824600.19.0.1700824600.60.0.0; _ga_GRDJMMMRHQ=GS1.2.1700824600.33.0.1700824601.59.0.0; _ga_GRSMPDQ6SE=GS1.2.1700824598.88.1.1700825516.60.0.0; _ga_M9ZHPEJXBM=GS1.2.1700824599.88.1.1700825517.60.0.0; qr-go-to-the-app-banner=1; sncid=b36ee46b-981d-11e5-add2-0050568f2465:1rBa43:O1Wy8KuwGDddSCM7NMHFsjGhdPaqxcjhdblvWWbVams; clt=new_old; _ym_isad=2; _gid=GA1.2.2131018395.1702274408; city_auto_popup_shown=1; _ym_visorc=b; _ga=GA1.2.333284150.1692163620; _ga_HJNSJ6NG5J=GS1.1.1702276831.120.1.1702280271.20.0.0; digi_uc=W1sidiIsIjM0OTMxNCIsMTcwMjI4MTIxMjA0Nl0sWyJ2IiwiODQwMjIiLDE3MDIyODAyMzEzNjZdLFsidiIsIjQ2Nzk3IiwxNzAyMjgwMTI4Mzc3XSxbInYiLCI5OTM3NTQiLDE3MDIyNzgwMTczMjddLFsidiIsIjE0NjUyMSIsMTcwMjI3NzI5NDExM10sWyJ2IiwiOTI2ODAiLDE3MDIyNzY5NDQ1NDRdLFsidiIsIjg4MjEzIiwxNzAxODU5MjE3OTg0XSxbInYiLCI5MTAyMSIsMTcwMTg1OTA2ODA2MF0sWyJ2IiwiOTg3MDUwIiwxNzAxNzc4NDc0NzY5XSxbInYiLCIyNTYyMTciLDE3MDEzNDIzNTc4NTRdXQ==',
    'Origin': 'https://spb.sunlight.net',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'X-Requested-With': 'SunlightFrontendApp',
    'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

# params = {
#     'article': '35517',
#     'city_id': '118',
# }

# response = requests.get(
#     'https://api.sunlight.net/v6/catalog/products/card/purchase/',
#     params=params,
#     cookies=cookies_discount,
#     headers=headers_discount,
# )

# Сбор скидок и цен

In [6]:
def get_personal_price(table):
    data_price = []
    for art in tqdm(range(len(table['article']))):
        # prx = choice(proxy_list)
        params = {
            'article': table['article'][art],
            'city_id': '118'
        }
        try:
            price_json = requests.get(
                'https://api.sunlight.net/v6/catalog/products/card/prices/',
                params=params,
                cookies=cookies_discount,
                headers=headers_discount,
                timeout=15,
                verify=False
        )
        except:
            time.sleep(6)
            price_json = requests.get(
                'https://api.sunlight.net/v6/catalog/products/card/prices/',
                params=params,
                cookies=cookies_discount,
                headers=headers_discount,
                timeout=15,
                verify=False
                            )
        try:
            discount_json = requests.get(
                'https://api.sunlight.net/v6/catalog/products/card/purchase/',
                params=params,
                cookies=cookies_discount,
                headers=headers_discount,
                timeout=15,
                verify=False
                            )
        except:
            time.sleep(6)
            discount_json = requests.get(
                'https://api.sunlight.net/v6/catalog/products/card/purchase/',
                params=params,
                cookies=cookies_discount,
                headers=headers_discount,
                timeout=15,
                verify=False
                            )
        discount = []
        try:
            price_old = price_json.json()['content']['default_price']['base_sum']
        except:
            price_old = np.nan
        try:
            price = price_json.json()['content']['default_price'][('result_sum')]
        except:
            price = np.nan
        try:
            price_personal = price_json.json()['content']['default_price']['personal_block']['value']
        except:
            price_personal = np.nan

        try:
            for i in range(len(discount_json.json()['content']['nameplates'])):
                discount.append(discount_json.json()['content']['nameplates'][i]['text']['label'])
        except:
            discount.append(np.nan)
            
        data_price.append({
            'article': table['article'][art],
            'price_old': price_old,
            'price': price,
            'price_personal': price_personal,
            'discount': discount

        })
    return data_price

# Сбор цены, названия, группы, артикула

In [7]:
def get_sunlight(soup):
    try:
        price = soup.find(class_="supreme-product-card__price-discount-price").text
        price = re.sub(r'\D', '', str(price))
    except:
        price = np.nan
    try:
        name_product = soup.find('h1', class_="supreme-product-card__about-title").text
    except:
        name_product = np.nan
    try:
        group_product = soup.find_all('span', {'property': 'name'})[1].text
    except:
        group_product = np.nan
    try:
        article_product = soup.find_all('div', class_="supreme-product-card__product-article")[0].find('span').text
        article_product = re.sub('\D', '', str(article_product))
    except:
        article_product = np.nan
    return price, name_product, group_product, article_product

# Сбор всех параметров словарем

In [8]:
def get_params(soup):
    names = []
    values = []
    leight = len(soup.find_all('span', {'class': 'supreme-product-card-description__item-text'}))
    list_names = soup.find_all('div', {'class': 'supreme-product-card-description__item-title'})[-leight:]
    for i in range(leight):
        try:
            nam = list_names[i].text
            names.append(nam.replace('\n', '').strip())
        except:
            names.append(np.nan)
        try:
            values.append(soup.find_all('span', {'class': 'supreme-product-card-description__item-text'})[i].text)
        except:
            values.append(np.nan)
    sunlight_params = dict(zip(names, values))
    return sunlight_params

# Сбор ссылок

In [20]:
def get_links(url_sunlight):
    home = 'https://sunlight.net'
    links = []
    pages_max = 250
    pages = int(pages_max + 1)
    for i in tqdm(range(1, pages)):
        url = f'{url_sunlight}&page={i}'
        resp = requests.get(url, headers={'User-Agent': UserAgent().chrome}, verify=False, timeout=70)
        soup = BeautifulSoup(resp.text, 'html.parser')
        for link in soup.find_all('div', class_="cl-item js-cl-item"):
            links.append(home + link.find('a', href=True)['href'])
    links = list(map(lambda x: x.replace('#product-reviews', ''), links))
    return links

In [21]:
# proxy = {'http': f'socks5://{choice(proxy_list)}','https': f'socks5://{choice(proxy_list)}'}
# print(proxy)

In [22]:
# proxy = choice(proxy_list)
# print(proxy)

# Общий сбор данных

In [23]:
def sunlight_parser(links):
    data = []
    click = 0
    for link_pars in tqdm(links):
        # prx = choice(proxy_list)
        try:
            resp = requests.get(link_pars, headers={'User-Agent': UserAgent().chrome}, timeout=70, verify=False)
        except:
            time.sleep(6)
            resp = requests.get(link_pars, headers={'User-Agent': UserAgent().chrome}, timeout=70, verify=False)
        good = BeautifulSoup(resp.text, 'lxml')
        data.append({"date": current_datetime,
                    "competitor": "Sunlight",
                    "name": get_sunlight(good)[1],
                    "group": get_sunlight(good)[2],
                    "params": get_params(good),
                    "price": get_sunlight(good)[0],
                    "article": get_sunlight(good)[3],
                    'url': link_pars
                    })
        if click == 50:
            click = 0
            time.sleep(0.5)
        else:
            continue
    data = pd.DataFrame(data)
    return data

In [24]:
links_sunlight = get_links('https://spb.sunlight.net/catalog/?proba=2494')

  0%|          | 0/250 [00:00<?, ?it/s]

In [27]:
b = pd.DataFrame(links_sunlight)

In [28]:
len(b.drop_duplicates())

14934

In [29]:
# Чтобы данные точно собрались нужно разделить ссылки на 100 чанков
chunks = np.array_split(links_sunlight, 100)

In [30]:
n = 0
for i in range(0, 100):
    n+=1
    print(f'начало парсинга {n} части ссылок')
    df = sunlight_parser(chunks[i])
    df.to_csv(r'C:\Users\Trenkin.Sergey\Desktop\parsers\sunlight_current_data\\' + f'sunlight_{day_now}_{month_now}_{n}.csv', index=False)
    time.sleep(5)

начало парсинга 1 части ссылок


  0%|          | 0/150 [00:00<?, ?it/s]


------------------------------------
начало парсинга 2 части ссылок


  0%|          | 0/150 [00:00<?, ?it/s]


------------------------------------
начало парсинга 3 части ссылок


...


------------------------------------
начало парсинга 99 части ссылок


  0%|          | 0/149 [00:00<?, ?it/s]


------------------------------------
начало парсинга 100 части ссылок


  0%|          | 0/149 [00:00<?, ?it/s]


------------------------------------


In [31]:
os.chdir(r'C:\Users\Trenkin.Sergey\Desktop\parsers\sunlight_current_data')
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
len(all_filenames)

100

In [32]:
data = pd.DataFrame()
for name  in all_filenames:
    df = pd.read_csv(name)
    data = pd.concat([df, data], ignore_index=True)

In [33]:
data

,date,competitor,name,group,params,price,article,url
0,14-03-2024,Sunlight,\n Золотой браслет с фианитами\n,Браслеты,"{'Модель': 'S-561018б_001*', 'Материал изделия...",9490,342484,https://sunlight.net/catalog/bracelets_342484....
1,14-03-2024,Sunlight,\n Золотое шейное украшение с сапфирами и б...,Шейное украшение,"{'Модель': '72768-H5W-01', 'Материал изделия':...",64990,342285,https://sunlight.net/catalog/neck_decoration_3...
2,14-03-2024,Sunlight,\n Золотое шейное украшение\n,Шейное украшение,"{'Модель': 'П40016462*', 'Материал изделия': '...",23690,341448,https://sunlight.net/catalog/neck_decoration_3...
3,14-03-2024,Sunlight,\n Золотое шейное украшение\n,Шейное украшение,"{'Модель': 'П10016462*', 'Материал изделия': '...",23230,341447,https://sunlight.net/catalog/neck_decoration_3...
4,14-03-2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': '77065', 'Материал изделия': 'Розов...",126030,340830,https://sunlight.net/catalog/bracelets_340830....
...,...,...,...,...,...,...,...,...
14991,14-03-2024,Sunlight,\n Золотая подвеска с бриллиантами\n,Подвески,"{'Модель': '73691-P5R-01', 'Материал изделия':...",21990,988887,https://sunlight.net/catalog/pendants_988887.html
14992,14-03-2024,Sunlight,\n Золотые серьги с фианитами\n,Серьги,"{'Модель': '3988К*', 'Материал изделия': 'Розо...",9190,347717,https://sunlight.net/catalog/earring_59995.html
14993,14-03-2024,Sunlight,\n Золотой крест-подвеска\n,Крест-подвеска,"{'Модель': '10029100*', 'Материал изделия': 'Р...",7490,74528,https://sunlight.net/catalog/pendants_74528.html
14994,14-03-2024,Sunlight,\n Золотое кольцо\n,Кольца,"{'Модель': '17-1001-11-61*', 'Покрытие': 'Без ...",11350,275727,https://sunlight.net/catalog/ring_275727.html


In [34]:
data.to_csv(rf'C:\Users\Trenkin.Sergey\Desktop\parsers\sunlight_current_data\data_without_price.csv', index = False)

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14996 entries, 0 to 14995
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        14996 non-null  object
 1   competitor  14996 non-null  object
 2   name        14996 non-null  object
 3   group       14996 non-null  object
 4   params      14996 non-null  object
 5   price       14996 non-null  int64 
 6   article     14996 non-null  int64 
 7   url         14996 non-null  object
dtypes: int64(2), object(6)
memory usage: 937.4+ KB


In [36]:
data.duplicated().sum()

62

In [37]:
data = data.drop_duplicates()

In [38]:
data.duplicated().sum()

0

In [39]:
data['article'].isna().sum()

0

In [40]:
data = data.dropna()

In [41]:
data['article'] = data['article'].astype('int') 

In [47]:
test = np.array_split(data, 100)

In [48]:
test[1].reset_index()

,index,date,competitor,name,group,params,price,article,url
0,150,14-03-2024,Sunlight,\n Золотое шейное украшение с сапфирами\n,Шейное украшение,"{'Модель': '73536-H5R-02', 'Материал изделия':...",34990,991191,https://sunlight.net/catalog/neck_decoration_9...
1,151,14-03-2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': '12893-B5Y-01', 'Материал изделия':...",54990,991138,https://sunlight.net/catalog/bracelets_991138....
2,152,14-03-2024,Sunlight,\n Золотое шейное украшение\n,Шейное украшение,"{'Модель': '12893-H5Y-01', 'Материал изделия':...",124990,991137,https://sunlight.net/catalog/neck_decoration_9...
3,153,14-03-2024,Sunlight,\n Золотое шейное украшение\n,Шейное украшение,"{'Модель': '00241-50-030001-00*', 'Материал из...",68260,991110,https://sunlight.net/catalog/neck_decoration_9...
4,154,14-03-2024,Sunlight,\n Золотое шейное украшение с фианитами\n,Шейное украшение,"{'Модель': '01Л132158*', 'Материал изделия': '...",25640,991016,https://sunlight.net/catalog/neck_decoration_9...
...,...,...,...,...,...,...,...,...,...
145,295,14-03-2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': '01Б013059Р*', 'Материал изделия': ...",73420,343091,https://sunlight.net/catalog/bracelets_343091....
146,296,14-03-2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': '01Б013043*', 'Материал изделия': '...",63180,343088,https://sunlight.net/catalog/bracelets_343088....
147,297,14-03-2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': '01Б012892*', 'Материал изделия': '...",92230,343083,https://sunlight.net/catalog/bracelets_343083....
148,298,14-03-2024,Sunlight,\n Золотые серьги с фианитами и кварцами ды...,Серьги,"{'Модель': 'кл2929а-4*', 'Материал изделия': '...",67990,332768,https://sunlight.net/catalog/earring_332768.html


In [44]:
n = 0
for i in range(0, 10):
    n+=1
    print(f'начало парсинга {i} части ссылок')
    
    df = get_personal_price(test[i].reset_index())
    df = pd.DataFrame(df)
    print('')
    print('------------------------------------')
    df.to_csv(r'C:\Users\Trenkin.Sergey\Desktop\parsers\sunlight_current_data\discounts\\' + f'sunlight_all_data_{day_now}_{month_now}_{n}.csv', index=False)
    time.sleep(5)

начало парсинга 1 части ссылок
начало парсинга 0 части ссылок


  0%|          | 0/1494 [00:00<?, ?it/s]


------------------------------------
начало парсинга 2 части ссылок
начало парсинга 1 части ссылок


  0%|          | 0/1494 [00:00<?, ?it/s]


------------------------------------
начало парсинга 3 части ссылок
начало парсинга 2 части ссылок


...


------------------------------------
начало парсинга 9 части ссылок
начало парсинга 8 части ссылок


  0%|          | 0/1493 [00:00<?, ?it/s]


------------------------------------
начало парсинга 10 части ссылок
начало парсинга 9 части ссылок


  0%|          | 0/1493 [00:00<?, ?it/s]


------------------------------------


In [49]:
os.chdir(r'C:\Users\Trenkin.Sergey\Desktop\parsers\sunlight_current_data\discounts')
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
len(all_filenames)

10

In [50]:
data

,date,competitor,name,group,params,price,article,url
0,14-03-2024,Sunlight,\n Золотой браслет с фианитами\n,Браслеты,"{'Модель': 'S-561018б_001*', 'Материал изделия...",9490,342484,https://sunlight.net/catalog/bracelets_342484....
1,14-03-2024,Sunlight,\n Золотое шейное украшение с сапфирами и б...,Шейное украшение,"{'Модель': '72768-H5W-01', 'Материал изделия':...",64990,342285,https://sunlight.net/catalog/neck_decoration_3...
2,14-03-2024,Sunlight,\n Золотое шейное украшение\n,Шейное украшение,"{'Модель': 'П40016462*', 'Материал изделия': '...",23690,341448,https://sunlight.net/catalog/neck_decoration_3...
3,14-03-2024,Sunlight,\n Золотое шейное украшение\n,Шейное украшение,"{'Модель': 'П10016462*', 'Материал изделия': '...",23230,341447,https://sunlight.net/catalog/neck_decoration_3...
4,14-03-2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': '77065', 'Материал изделия': 'Розов...",126030,340830,https://sunlight.net/catalog/bracelets_340830....
...,...,...,...,...,...,...,...,...
14991,14-03-2024,Sunlight,\n Золотая подвеска с бриллиантами\n,Подвески,"{'Модель': '73691-P5R-01', 'Материал изделия':...",21990,988887,https://sunlight.net/catalog/pendants_988887.html
14992,14-03-2024,Sunlight,\n Золотые серьги с фианитами\n,Серьги,"{'Модель': '3988К*', 'Материал изделия': 'Розо...",9190,347717,https://sunlight.net/catalog/earring_59995.html
14993,14-03-2024,Sunlight,\n Золотой крест-подвеска\n,Крест-подвеска,"{'Модель': '10029100*', 'Материал изделия': 'Р...",7490,74528,https://sunlight.net/catalog/pendants_74528.html
14994,14-03-2024,Sunlight,\n Золотое кольцо\n,Кольца,"{'Модель': '17-1001-11-61*', 'Покрытие': 'Без ...",11350,275727,https://sunlight.net/catalog/ring_275727.html


In [51]:
data_discount = pd.DataFrame()
for name  in all_filenames:
    df = pd.read_csv(name)
    data_discount = pd.concat([df, data_discount], ignore_index=True)

In [52]:
data = data.merge(data_discount, on='article', how='left')

In [53]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14934 entries, 0 to 14933
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   date            14934 non-null  object
 1   competitor      14934 non-null  object
 2   name            14934 non-null  object
 3   group           14934 non-null  object
 4   params          14934 non-null  object
 5   price_x         14934 non-null  int64 
 6   article         14934 non-null  int32 
 7   url             14934 non-null  object
 8   price_old       14201 non-null  object
 9   price_y         14934 non-null  object
 10  price_personal  14292 non-null  object
 11  discount        14934 non-null  object
dtypes: int32(1), int64(1), object(10)
memory usage: 1.3+ MB


In [54]:
data['discount'].value_counts()

discount
['Последний день! ЕЩЕ -30%, код: ЛЮБИ']            4795
['Последний день! ЕЩЕ -35%, код: ЛЮБИ']            3963
['Последний день! ЕЩЕ -40%, код: ЛЮБИ']            3059
['Последний день! ЕЩЕ -20%, код: ЛЮБИ']            2187
['ШОК! Всего 4 990 руб. за грамм!', 'ХИТ-цена']     453
[nan]                                               244
['ХИТ-цена']                                        182
['ЕЩЕ -35% по коду: СЕГОДНЯ']                        51
Name: count, dtype: int64

In [55]:
data.head(20)

,date,competitor,name,group,params,price_x,article,url,price_old,price_y,price_personal,discount
0,14-03-2024,Sunlight,\n Золотой браслет с фианитами\n,Браслеты,"{'Модель': 'S-561018б_001*', 'Материал изделия...",9490,342484,https://sunlight.net/catalog/bracelets_342484....,18 980 ₽,9 490 ₽,6 643 ₽,"['Последний день! ЕЩЕ -30%, код: ЛЮБИ']"
1,14-03-2024,Sunlight,\n Золотое шейное украшение с сапфирами и б...,Шейное украшение,"{'Модель': '72768-H5W-01', 'Материал изделия':...",64990,342285,https://sunlight.net/catalog/neck_decoration_3...,129 980 ₽,64 990 ₽,47 990 ₽,"['Последний день! ЕЩЕ -20%, код: ЛЮБИ']"
2,14-03-2024,Sunlight,\n Золотое шейное украшение\n,Шейное украшение,"{'Модель': 'П40016462*', 'Материал изделия': '...",23690,341448,https://sunlight.net/catalog/neck_decoration_3...,47 380 ₽,23 690 ₽,16 583 ₽,"['Последний день! ЕЩЕ -30%, код: ЛЮБИ']"
3,14-03-2024,Sunlight,\n Золотое шейное украшение\n,Шейное украшение,"{'Модель': 'П10016462*', 'Материал изделия': '...",23230,341447,https://sunlight.net/catalog/neck_decoration_3...,46 460 ₽,23 230 ₽,16 261 ₽,"['Последний день! ЕЩЕ -30%, код: ЛЮБИ']"
4,14-03-2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': '77065', 'Материал изделия': 'Розов...",126030,340830,https://sunlight.net/catalog/bracelets_340830....,252 060 ₽,126 030 ₽,109 030 ₽,"['Последний день! ЕЩЕ -20%, код: ЛЮБИ']"
5,14-03-2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': '77061', 'Материал изделия': 'Розов...",100810,340829,https://sunlight.net/catalog/bracelets_340829....,201 620 ₽,100 810 ₽,83 810 ₽,"['Последний день! ЕЩЕ -20%, код: ЛЮБИ']"
6,14-03-2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': '77060', 'Материал изделия': 'Розов...",118100,340828,https://sunlight.net/catalog/bracelets_340828....,236 200 ₽,118 100 ₽,101 100 ₽,"['Последний день! ЕЩЕ -20%, код: ЛЮБИ']"
7,14-03-2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': '77022', 'Материал изделия': 'Розов...",151720,340827,https://sunlight.net/catalog/bracelets_340827....,303 440 ₽,151 720 ₽,134 720 ₽,"['Последний день! ЕЩЕ -20%, код: ЛЮБИ']"
8,14-03-2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': 'Т3010587240*', 'Материал изделия':...",18540,340189,https://sunlight.net/catalog/bracelets_340189....,37 080 ₽,18 540 ₽,12 978 ₽,"['Последний день! ЕЩЕ -35%, код: ЛЮБИ']"
9,14-03-2024,Sunlight,\n Золотое обручальное кольцо \n,Кольца,"{'Модель': 'Т10001302*', 'Материал изделия': '...",16950,339930,https://sunlight.net/catalog/ring_339930.html,33 900 ₽,16 950 ₽,11 865 ₽,"['Последний день! ЕЩЕ -40%, код: ЛЮБИ']"


In [57]:
data.to_csv(r'C:\Users\Trenkin.Sergey\Desktop\parsers\sunlight_current_data\\' + f'sunlight_all_{day_now}_{month_now}.csv', index=False)

In [58]:
data.to_csv(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\parsed_data\sunlight_all_{day_now}_{month_now}_{datetime.today().year}.csv', index=False)